# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("../WeatherPy/cities.csv")
city_df = city_df.dropna()
city_df.head()

,city,cloudiness,country,date,humidity,lat,lng,temp,windspeed
0,grindavik,75.0,IS,1.596469e+09,81.0,63.84,-22.43,50.00,8.05
1,busselton,84.0,AU,1.596469e+09,76.0,-33.65,115.33,48.99,8.81
2,los llanos de aridane,20.0,ES,1.596469e+09,69.0,28.66,-17.92,77.00,19.46
3,reguengos de monsaraz,20.0,PT,1.596469e+09,27.0,38.43,-7.53,97.00,17.22
5,vila velha,75.0,BR,1.596469e+09,57.0,-20.33,-40.29,75.20,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["lat","lng"]]
humidity = city_df["humidity"]

In [4]:
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10,point_radius=1)

#Adding to figure
fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
fit_weather = city_df.loc[(city_df["temp"] < 75) 
                          &(city_df["temp"] > 65) 
                          &(city_df["windspeed"] < 12) 
                          &(city_df["cloudiness"] < 25)
                         ]
fit_weather.head()

,city,cloudiness,country,date,humidity,lat,lng,temp,windspeed
11,aykhal,0.0,RU,1.596469e+09,45.0,66.00,111.50,66.20,6.71
15,cerritos,20.0,US,1.596469e+09,73.0,33.86,-118.06,73.99,2.82
53,castro,16.0,BR,1.596469e+09,47.0,-24.79,-50.01,67.87,3.20
57,williston,0.0,US,1.596469e+09,55.0,48.15,-103.62,71.01,5.01
63,leh,3.0,IN,1.596469e+09,23.0,34.17,77.58,68.23,6.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(fit_weather)
hotel_df['Hotel Name'] = ""
hotel_df.head()

,city,cloudiness,country,date,humidity,lat,lng,temp,windspeed,Hotel Name
11,aykhal,0.0,RU,1.596469e+09,45.0,66.00,111.50,66.20,6.71,
15,cerritos,20.0,US,1.596469e+09,73.0,33.86,-118.06,73.99,2.82,
53,castro,16.0,BR,1.596469e+09,47.0,-24.79,-50.01,67.87,3.20,
57,williston,0.0,US,1.596469e+09,55.0,48.15,-103.62,71.01,5.01,
63,leh,3.0,IN,1.596469e+09,23.0,34.17,77.58,68.23,6.15,


In [7]:
#Google Places API call
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    lat_lng = str(lat) + ", " + str(lng)
    distance = 5000
    city_name = row['city']
    
    params = {
    "location": lat_lng,  
    "radius": distance,
    "type": "hotel",
    "key": g_key,
    }
    #Making the API call in loop
    res = requests.get(url,params).json()
    results = res['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError):
        print("Hotel not found. Skipping...")

In [8]:
hotel_df.head(20)

,city,cloudiness,country,date,humidity,lat,lng,temp,windspeed,Hotel Name
11,aykhal,0.0,RU,1.596469e+09,45.0,66.00,111.50,66.20,6.71,Aikhal
15,cerritos,20.0,US,1.596469e+09,73.0,33.86,-118.06,73.99,2.82,Long Beach
53,castro,16.0,BR,1.596469e+09,47.0,-24.79,-50.01,67.87,3.20,Castro
57,williston,0.0,US,1.596469e+09,55.0,48.15,-103.62,71.01,5.01,Williston
63,leh,3.0,IN,1.596469e+09,23.0,34.17,77.58,68.23,6.15,Leh
68,zhangjiakou,9.0,CN,1.596469e+09,53.0,40.81,114.88,71.06,3.00,Zhangjiakou
71,sakura,20.0,JP,1.596469e+09,94.0,35.72,140.23,73.99,2.24,Sakura
74,hasaki,20.0,JP,1.596469e+09,94.0,35.73,140.83,73.99,2.24,Kamisu
84,kruisfontein,11.0,ZA,1.596469e+09,64.0,-34.00,24.73,66.00,11.88,Humansdorp
115,yuanping,0.0,CN,1.596469e+09,40.0,38.72,112.76,70.09,2.55,Xinzhou


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))